<a href="https://colab.research.google.com/github/dhirajs16/Code_Basics_Practice/blob/main/Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# /content/drive/MyDrive/TB_Chest_Radiography_Database/Normal/Normal-1.png
# /content/drive/MyDrive/TB_Chest_Radiography_Database/Tuberculosis/Tuberculosis-1.png

In [ ]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the dataset
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (128, 128))
            images.append(img)
    return np.array(images)

# Load the dataset according to the new folder structure
def load_dataset():
    normal_images = load_images_from_folder('/content/drive/MyDrive/TB_Chest_Radiography_Database/Normal')
    tb_images = load_images_from_folder('/content/drive/MyDrive/TB_Chest_Radiography_Database/Tuberculosis')

    normal_images = normal_images / 255.0  # Normalize the images
    tb_images = tb_images / 255.0  # Normalize the images

    normal_images = np.expand_dims(normal_images, axis=-1)  # Add channel dimension
    tb_images = np.expand_dims(tb_images, axis=-1)  # Add channel dimension

    return normal_images, tb_images

healthy_images, _ = load_dataset()

# Define the autoencoder
input_img = tf.keras.Input(shape=(128, 128, 1))
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
autoencoder = models.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the autoencoder
autoencoder.fit(healthy_images, healthy_images, epochs=50, batch_size=32, shuffle=True, validation_split=0.2)

def detect_anomalies(images, model, threshold):
    reconstructed = model.predict(images)
    reconstruction_error = np.mean(np.square(images - reconstructed), axis=(1, 2, 3))
    anomalies = reconstruction_error > threshold
    return anomalies, reconstruction_error

# Load test images (both normal and tuberculosis)
normal_test_images = load_images_from_folder('/content/drive/MyDrive/TB_Chest_Radiography_Database/Normal')
tb_test_images = load_images_from_folder('/content/drive/MyDrive/TB_Chest_Radiography_Database/Tuberculosis')

# Normalize the images
normal_test_images = normal_test_images / 255.0
tb_test_images = tb_test_images / 255.0

# Add channel dimension
normal_test_images = np.expand_dims(normal_test_images, axis=-1)
tb_test_images = np.expand_dims(tb_test_images, axis=-1)

# Concatenate the images along a new axis
test_images = np.concatenate((normal_test_images, tb_test_images), axis=0)

# Detect anomalies
threshold = 0.01  # Set a threshold based on your model's performance
anomalies, reconstruction_error = detect_anomalies(test_images, autoencoder, threshold)

# Print results
for i, is_anomaly in enumerate(anomalies):
    print(f"Image {i}: {'Anomaly' if is_anomaly else 'Normal'}, Reconstruction Error: {reconstruction_error[i]}")


Epoch 1/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - loss: 0.6170 - val_loss: 0.5456
Epoch 2/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.5444 - val_loss: 0.5405
Epoch 3/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.5414 - val_loss: 0.5390
Epoch 4/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.5391 - val_loss: 0.5380
Epoch 5/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.5387 - val_loss: 0.5375
Epoch 6/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.5381 - val_loss: 0.5375
Epoch 7/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.5371 - val_loss: 0.5371
Epoch 8/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.5373 - val_loss: 0.5370
Epoch 9/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.5369 - val_loss: 0.5368
Epoch 10/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.5367 - val_loss: 0.5367
Epoch 11/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.5368 - val_loss: 0.5366
Epoch 12/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.5

In [ ]:
# Save the model
autoencoder.save('/content/drive/MyDrive/TB_Chest_Radiography_Database/tb_anomaly_detector_model.h5')


In [ ]:
# Save the model
autoencoder.save('/content/drive/MyDrive/TB_Chest_Radiography_Database/tb_anomaly_detector_model.keras')


In [1]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the dataset
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (128, 128))
            images.append(img)
    return np.array(images)

# Load the dataset according to the new folder structure
def load_dataset():
    normal_images = load_images_from_folder('/content/drive/MyDrive/TB_Chest_Radiography_Database/Normal')

    normal_images = normal_images / 255.0  # Normalize the images
    normal_images = np.expand_dims(normal_images, axis=-1)  # Add channel dimension

    return normal_images

healthy_images = load_dataset()

# Define the autoencoder
input_img = tf.keras.Input(shape=(128, 128, 1))
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
autoencoder = models.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the autoencoder
autoencoder.fit(healthy_images, healthy_images, epochs=50, batch_size=32, shuffle=True, validation_split=0.2)

def detect_anomalies(images, model, threshold):
    reconstructed = model.predict(images)
    reconstruction_error = np.mean(np.square(images - reconstructed), axis=(1, 2, 3))
    anomalies = reconstruction_error > threshold
    return anomalies, reconstruction_error

# Load test images (both normal and tuberculosis)
normal_test_images = load_images_from_folder('/content/drive/MyDrive/TB_Chest_Radiography_Database/Normal')
tb_test_images = load_images_from_folder('/content/drive/MyDrive/TB_Chest_Radiography_Database/Tuberculosis')

# Normalize the images
normal_test_images = normal_test_images / 255.0
tb_test_images = tb_test_images / 255.0

# Add channel dimension
normal_test_images = np.expand_dims(normal_test_images, axis=-1)
tb_test_images = np.expand_dims(tb_test_images, axis=-1)

# Concatenate the images along a new axis
test_images = np.concatenate((normal_test_images, tb_test_images), axis=0)

# Detect anomalies
threshold = 0.01  # Set a threshold based on your model's performance
anomalies, reconstruction_error = detect_anomalies(test_images, autoencoder, threshold)

# Print results
for i, is_anomaly in enumerate(anomalies):
    print(f"Image {i}: {'Anomaly' if is_anomaly else 'Normal'}, Reconstruction Error: {reconstruction_error[i]}")


Epoch 1/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 12s 57ms/step - loss: 0.6220 - val_loss: 0.5441
Epoch 2/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.5416 - val_loss: 0.5401
Epoch 3/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.5402 - val_loss: 0.5390
Epoch 4/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.5390 - val_loss: 0.5385
Epoch 5/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.5391 - val_loss: 0.5378
Epoch 6/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.5387 - val_loss: 0.5375
Epoch 7/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.5388 - val_loss: 0.5373
Epoch 8/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.5383 - val_loss: 0.5370
Epoch 9/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.5379 - val_loss: 0.5377
Epoch 10/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.5368 - val_loss: 0.5370
Epoch 11/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.5372 - val_loss: 0.5366
Epoch 12/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.

In [2]:
autoencoder.save('/content/drive/MyDrive/TB_Chest_Radiography_Database/tb_anomaly_detector_model2.keras')
